# Masterthesis
#### Julian Jetz

In [1]:
import pandas as pd
import numpy as np
import tensorflow tf
from tensorflow.python.data import Dataset

from scipy import stats
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from pandas.plotting import scatter_matrix
from currency_converter import CurrencyConverter
from datetime import date


SyntaxError: invalid syntax (<ipython-input-1-d642765712c8>, line 3)

In [ ]:
na_values = ['nan', 'N/A', 'NaN', 'NaT']

In [ ]:
df = pd.read_csv('resources/accident_data.csv', sep=";", na_values=na_values, index_col=False, dtype = {"STATE" : "str", "TYPE" : "str", "TYPEQ" : "str", "WEATHER" : "str", "VISIBLTY" : "str"})
df.shape[0]

In [ ]:
std_dev = 2
df = df[(np.abs(stats.zscore(df[['LOCOMOTIVES1','LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOCOMOTIVES2','LOADF2', 'EMPTYF2', 'INFRASTRUCTURE_DMG']])) < float(std_dev)).all(axis=1)]
df.shape[0]

In [ ]:
pd.set_option('display.max_columns', 500)
#df.head(50)

In [2]:
df[df['ACCIDENT_TYPE'].isnull()].shape[0]

NameError: name 'df' is not defined

In [ ]:
df[df['EQUIPMENT_TYPE'].isnull()].shape[0]

In [ ]:
df[df['WEATHER'].isnull()].shape[0]

In [ ]:
df[df['VISIBLTY'].isnull()].shape[0]

In [ ]:
df[df['Jahreszeit'].isnull()].shape[0]

In [ ]:
df[df['TONS']==0].shape[0]

In [ ]:
df[df['SPEED']==0].shape[0]

In [ ]:
df.head(50)

#### [Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) <br>
Die Adam-Optimierung ist eine stochastische Gradientenabsenkungsmethode, die auf einer adaptiven Schätzung von Momenten erster und zweiter Ordnung basiert. Das Verfahren ist "recheneffizient, hat wenig Speicherbedarf, ist invariant gegenüber der diagonalen Neuskalierung von Gradienten und eignet sich gut für Probleme, die in Bezug auf Daten/Parameter groß sind".
[Arxiv](https://arxiv.org/pdf/1412.6980.pdf)<br><br>
*Learning rate:* In der maschinellen Lern- und Statistiktechnik ist die Lernrate ein Tuningparameter in einem Optimierungsalgorithmus, der die Schrittweite bei jeder Iteration bestimmt und sich dabei auf ein Minimum einer Verlustfunktion zubewegt. Da sie beeinflusst, inwieweit neu gewonnene Informationen alte Informationen übersteuern, stellt sie metaphorisch die Geschwindigkeit dar, mit der ein maschinelles Lernmodell "lernt". Bei der Festlegung einer Lernrate gibt es einen Kompromiss zwischen der Konvergenzrate und der Überschreitung. Während die Richtung zum Minimum in der Regel aus dem Gradienten der Verlustfunktion bestimmt wird, bestimmt die Lernrate, wie groß ein Schritt in diese Richtung ist.Eine zu hohe Lernrate führt dazu, dass der Lernsprung über Minima hinausgeht, aber eine zu niedrige Lernrate dauert entweder zu lange, um sich zu konvergieren oder in einem unerwünschten lokalen Minimum stecken zu bleiben.


In [ ]:
opti = tf.train.AdamOptimizer()

## Complete regression model

Festlegen der abhängigen Variablen X und der vorherzusagenden Variable y

In [ ]:
x_data = df[['YEAR4', 'MONTH', 'LOCOMOTIVES1', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1','LOCOMOTIVES2', 'LOADF2', 'EMPTYF2', 'TONS', 'TEMP_CLUSTER', 'SPEED', 'ACCCAUSE_LVL1', 'TYPE', 'EQUIPMENT_TYPE', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'STATE', 'SPD_TOO_HIGH', 'ACCTYPE', 'ACCIDENT_TYPE', 'Jahreszeit']]
y = df['INFRASTRUCTURE_DMG']

#### Aufteilen der Daten in Train und Test Datensatz

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y, test_size=0.3)

In [ ]:
x_data.dtypes

#### Festlegen der numerischen Merkmalsspalten

In [ ]:
trnspd = tf.feature_column.numeric_column('SPEED')
year = tf.feature_column.numeric_column('YEAR4')
locomotives1 = tf.feature_column.numeric_column('LOCOMOTIVES1')
loadf1 = tf.feature_column.numeric_column('LOADF1')
loadp1 = tf.feature_column.numeric_column('LOADP1')
emptyf1 = tf.feature_column.numeric_column('EMPTYF1')
emptyp1 = tf.feature_column.numeric_column('EMPTYP1')
locomotives2 = tf.feature_column.numeric_column('LOCOMOTIVES2')
loadf2 = tf.feature_column.numeric_column('LOADF2')
emptyf2 = tf.feature_column.numeric_column('EMPTYF2')
tons = tf.feature_column.numeric_column('TONS')

In [ ]:
x_data[['ACCCAUSE_LVL1', 'TYPE', 'EQUIPMENT_TYPE', 'TYPTRK', 'VISIBLTY', 'WEATHER', 'STATE', 'SPD_TOO_HIGH', 'ACCTYPE', 'ACCIDENT_TYPE', 'TEMP_CLUSTER', 'Jahreszeit']].astype(str).values

#### Festlegen der kategorischen Merkmalsspalten. 
Anstatt die Daten als einen one-hot Vektor mit vielen Dimensionen darzustellen, stellt eine Einbettungsspalte diese Daten als einen niederdimensionalen, dichten Vektor dar, in dem jede Zelle eine beliebige Zahl enthalten kann, nicht nur 0 oder 1. Die Größe der Einbettung ist ein Parameter, der angepasst werden muss (*TODO*).

In [ ]:
accause = tf.feature_column.categorical_column_with_identity('ACCCAUSE_LVL1',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['ACCCAUSE_LVL1'].unique())**0.25))
accause=tf.feature_column.embedding_column(accause, dimension=embedding_size+1)

In [ ]:
accidenttype = tf.feature_column.categorical_column_with_identity('ACCIDENT_TYPE',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['ACCIDENT_TYPE'].unique())**0.25))
accidenttype=tf.feature_column.embedding_column(accidenttype, dimension=embedding_size)

In [ ]:
typeq = tf.feature_column.categorical_column_with_identity('EQUIPMENT_TYPE',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['EQUIPMENT_TYPE'].unique())**0.25))
typeq=tf.feature_column.embedding_column(typeq, dimension=embedding_size)

In [ ]:
typtrk = tf.feature_column.categorical_column_with_identity('TYPTRK',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['TYPTRK'].unique())**0.25))
typtrk=tf.feature_column.embedding_column(typtrk, dimension=embedding_size)

In [ ]:
visibility = tf.feature_column.categorical_column_with_identity('VISIBLTY',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['VISIBLTY'].unique())**0.25))
visibility=tf.feature_column.embedding_column(visibility, dimension=embedding_size)

In [ ]:
weather = tf.feature_column.categorical_column_with_identity('WEATHER',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['WEATHER'].unique())**0.25))
weather=tf.feature_column.embedding_column(weather, dimension=embedding_size)

In [ ]:
state = tf.feature_column.categorical_column_with_identity('STATE',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['STATE'].unique())**0.25))
state=tf.feature_column.embedding_column(state, dimension=embedding_size)

In [ ]:
jahreszeit = tf.feature_column.categorical_column_with_identity('Jahreszeit',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['Jahreszeit'].unique())**0.25))
jahreszeit=tf.feature_column.embedding_column(state, dimension=embedding_size)

In [ ]:
temp_cluster = tf.feature_column.categorical_column_with_identity('TEMP_CLUSTER',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['TEMP_CLUSTER'].unique())**0.25))
temp_cluster=tf.feature_column.embedding_column(temp_cluster, dimension=embedding_size)

In [ ]:
spd_too_high = tf.feature_column.categorical_column_with_identity('SPD_TOO_HIGH',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['SPD_TOO_HIGH'].unique())**0.25))
spd_too_high=tf.feature_column.embedding_column(spd_too_high, dimension=embedding_size)

In [ ]:
acctype = tf.feature_column.categorical_column_with_identity('ACCTYPE',num_buckets=100000)

embedding_size = int(math.floor(len(x_data['ACCTYPE'].unique())**0.25))
acctype=tf.feature_column.embedding_column(acctype, dimension=embedding_size)

#### Festlegen der Merkmalsspalten

In [ ]:
feature_col =[year, trnspd, typeq, locomotives1, loadf1, loadp1, emptyf1, emptyp1, locomotives2, loadf2, emptyf2, typtrk, accidenttype, accause, visibility, weather, temp_cluster, state, jahreszeit, spd_too_high, acctype]

#### Aufstellen der Input Funktion

Batch_Size=Größe der zurückzusendenden Batches.<br>
Num_Epochs=Anzahl der Perioden, die man über Daten iterieren muss.<br>
Shuffle=Sollendie Datensätze in zufälliger Reihenfolge gelesen werden?

In [ ]:
#input_func= tf.estimator.inputs.pandas_input_fn(x=x_train, y= y_train, batch_size=10, num_epochs=1000, shuffle=True)

In [ ]:
tf.enable_eager_execution()

In [ ]:
def input_fun(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    features = {key:np.array(value) for key,value in dict(features).items()}                                             
 
    # Construct a dataset, and configure batching/repeating.
    shuffleset = tf.data.Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    shuffleset = shuffleset.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
        shuffleset = shuffleset.shuffle(10000)
    
    # Return the next batch of data.
    iterator = shuffleset.__iter__()
    next_element = iterator.get_next()
    
    features, labels = next_element
    return features, labels
    

In [ ]:
input_func = lambda: input_fun(features=x_train, targets=y_train, num_epochs=1, shuffle=False)

#### Aufstellen der Eval Input Funktion

In [ ]:
test_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,                                                   
                                                 batch_size=10, 
                                                 num_epochs=1, 
                                                 shuffle=False)
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,
                                                      y=y_test, 
                                                      batch_size=10, 
                                                      num_epochs=1, 
                                                      shuffle=False)
train_input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,                                                   
                                                 batch_size=10, 
                                                 num_epochs=1, 
                                                 shuffle=False)

#### Initialisierung des Estimators (DNNRegressor)
hidden_units=Das Argument hidden_units ermöglicht es, ein Array mit der Anzahl der Knoten für jede Schicht zu erzeugen. Dies ermöglicht es, ein neuronales Netzwerk zu erstellen, indem einfach seine Größe und Form berücksichtigt wird, anstatt das Ganze von Grund auf von Hand zu vernetzen. (TODO: fine tune)

In [ ]:
#estimator = tf.estimator.DNNRegressor(hidden_units=[1024, 512, 256], feature_columns=feature_col, optimizer=lambda: tf.keras.optimizers.Adam(learning_rate=tf.compat.v1.train.exponential_decay(learning_rate=0.1,global_step=tf.compat.v1.train.get_global_step(),decay_steps=10000,decay_rate=0.96)))

In [ ]:
estimator = tf.estimator.DNNRegressor(hidden_units=[1024, 512, 256], feature_columns=feature_col)

#### Train Model

In [ ]:
estimator.train(input_fn=input_func, max_steps=60000)


#### Evaluate Model mit Eval input function

In [ ]:
result_eval = estimator.evaluate(input_fn=eval_input_func)
result_eval

#### Scatterplot Vergleich tatsächliche und vorhergesagte Werte

In [ ]:
predictions=[]
for pred in estimator.predict(input_fn=test_input_func):
    predictions.append(pred['predictions'][0].astype(float))
plt.plot(y_test, predictions, 'o')
plt.xlabel('Actual values (test data)')
plt.ylabel('predicted values (test data)')

In [ ]:
train_predictions=[]
for pred in estimator.predict(input_fn=train_input_func):
    train_predictions.append(pred['predictions'][0].astype(float))
plt.plot(y_train, train_predictions, 'o')
plt.xlabel('Actual values (train data)')
plt.ylabel('predicted values (train data)')

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))**0.5
rmse

#### Verleich tatsächliche und vorhergesagte Werte 
Beispiel: 30 zufällig ausgewählte Werte

In [ ]:
pred = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
pred1 = pred.sample(100)

pred1.plot(kind='bar',figsize=(20,16))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

#### Residual Plot 
Abweichung zwischen den vorhergesagten und tatsächlichen Unfallkosten (Testdaten=grün, Trainingsdaten=blau)

In [ ]:
    plt.scatter(train_predictions, train_predictions - y_train, c='b', s=40, alpha=0.5, label='Train Data')
    plt.scatter(predictions, predictions - y_test, c='g', s=40, label='Test Data')
    plt.hlines(y=0, xmin=-0.03, xmax=0.2)
    plt.title('Residual Plot of DNN Regression')
    plt.ylabel('Residuals')
    plt.xlabel('Accident Damage')
    plt.legend()
    plt.show()